In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

import numpy as np

seed = 7

Using TensorFlow backend.


## Hyperparameter runing on mobilenet with RMSprop.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15035409360860166317
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11067257652
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12203320788586079880
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
dir(keras.applications.mobilenet_v2)

['MobileNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'keras_modules_injection',
 'mobilenet_v2',
 'preprocess_input',
 'print_function']

In [4]:
np.random.seed(seed)

In [5]:
data_dir = '/data/oxford102/train'
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
img_width, img_height = 256, 256
batch_size = 128
epochs = 100
nr_categories = 102
nb_train_samples = 4604
nb_validation_samples = 1094 

In [6]:
def get_image_generator(input_processor, img_aug=False):
    if not img_aug:
        train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)
    else: 
        train_val_datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            preprocessing_function=input_processor,
            validation_split=0.2)
        
    return train_val_datagen

In [7]:
def get_generators(batch_size, image_size, input_processor, img_aug=False):

    img_width, img_height = image_size
    
    train_val_datagen = get_image_generator(input_processor, img_aug)

    train_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            batch_size=batch_size,
            subset="training",
            class_mode='categorical')

    validation_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            subset="validation",
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator


In [8]:
#from keras import regularizers

def get_model(network_name="inception_resnet_v2", image_size=(256, 256), verbose=False):
    k.set_learning_phase(0)

    img_width, img_height = image_size
    if network_name == "vgg16":
        base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg16.preprocess_input
    elif network_name == "vgg19":
        base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg19.preprocess_input
    elif network_name == "inception_resnet_v2":
        base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.inception_resnet_v2.preprocess_input
    elif network_name == "mobilenet_v2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.mobilenet_v2.preprocess_input
    else:
        raise Exception("check your network name")

    for layer in base_model.layers[:]:
        layer.trainable = False

        #Adding custom Layers 
    k.set_learning_phase(1)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu", 
              #kernel_regularizer=regularizers.l2(0.01),
             #       activity_regularizer=regularizers.l1(0.001)
             )(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x, training=True)
    #x = Dense(102, activation="relu")(x)
    predictions = Dense(nr_categories, activation="softmax")(x)

    _model = Model(input = base_model.input, output = predictions)
    if verbose:
        _model.summary()
    return _model, input_processor

In [10]:
# Hyperparameter optimization

In [ ]:
optimizers

In [ ]:
optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

In [ ]:
optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

In [ ]:
optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

In [10]:
#import talos as ta

# p = {'lr': (0.001, 0.01, 0.1),
#      'batch_size': (128),
#      'epochs': [150],
#      'dropout': (0, 0.5, 5),
#      'weight_regulizer':[None],
#      'emb_output_dims': [None],
#      'shape':['brick','long_funnel'],
#      'optimizer': [Adam, Nadam, RMSprop],
#      'losses': [logcosh, binary_crossentropy],
#      'activation':[relu, elu],
#      'last_activation': [sigmoid]}

In [9]:
def train_model(params, _model, generators):
    model_name = params["network_name"]
    num_train_img = 4604
    num_val_img = 1094

    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")

    train_generator, validation_generator = generators
    _model.compile(loss = "categorical_crossentropy", optimizer = params["optimizer"], metrics=["accuracy"])


    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = _model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // params["batch_size"] // 2,
                epochs=params["epochs"],
                validation_data=validation_generator,
                validation_steps=num_val_img // params["batch_size"],
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        params.pop("optimizer")
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params

# MobileNet + RMSPROP

In [1]:
import itertools
import numpy

In [2]:
lr = [0.1, 0.01, 0.001]
rho = [0.5, 0.75, 0.9]
epsilon = [1e-7, 1e-6, 1e-4, 1e-2]
decay = [0.001, 0.01, 0.1, 0.2]


In [3]:
# from keras import backend as K
# K.epsilon()
# >>> 1e-7

In [4]:
grid = list(itertools.product(lr, rho, epsilon, decay))

In [5]:
grid

[(0.1, 0.5, None, 0.001),
 (0.1, 0.5, None, 0.01),
 (0.1, 0.5, None, 0.1),
 (0.1, 0.5, None, 0.2),
 (0.1, 0.5, 1e-06, 0.001),
 (0.1, 0.5, 1e-06, 0.01),
 (0.1, 0.5, 1e-06, 0.1),
 (0.1, 0.5, 1e-06, 0.2),
 (0.1, 0.5, 0.0001, 0.001),
 (0.1, 0.5, 0.0001, 0.01),
 (0.1, 0.5, 0.0001, 0.1),
 (0.1, 0.5, 0.0001, 0.2),
 (0.1, 0.5, 0.01, 0.001),
 (0.1, 0.5, 0.01, 0.01),
 (0.1, 0.5, 0.01, 0.1),
 (0.1, 0.5, 0.01, 0.2),
 (0.1, 0.75, None, 0.001),
 (0.1, 0.75, None, 0.01),
 (0.1, 0.75, None, 0.1),
 (0.1, 0.75, None, 0.2),
 (0.1, 0.75, 1e-06, 0.001),
 (0.1, 0.75, 1e-06, 0.01),
 (0.1, 0.75, 1e-06, 0.1),
 (0.1, 0.75, 1e-06, 0.2),
 (0.1, 0.75, 0.0001, 0.001),
 (0.1, 0.75, 0.0001, 0.01),
 (0.1, 0.75, 0.0001, 0.1),
 (0.1, 0.75, 0.0001, 0.2),
 (0.1, 0.75, 0.01, 0.001),
 (0.1, 0.75, 0.01, 0.01),
 (0.1, 0.75, 0.01, 0.1),
 (0.1, 0.75, 0.01, 0.2),
 (0.1, 0.9, None, 0.001),
 (0.1, 0.9, None, 0.01),
 (0.1, 0.9, None, 0.1),
 (0.1, 0.9, None, 0.2),
 (0.1, 0.9, 1e-06, 0.001),
 (0.1, 0.9, 1e-06, 0.01),
 (0.1, 0.9, 1e-0

In [6]:
len(grid)

144

In [15]:
choices = numpy.random.choice(range(len(grid)), int(len(grid)*0.2), replace=False)

In [16]:
# choices
# >>> array([ 11, 122,  37,  32,  70, 113,  77, 116, 134,  74, 143,  66,  41,
#         81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
#         40,  45])

array([ 11, 122,  37,  32,  70, 113,  77, 116, 134,  74, 143,  66,  41,
        81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
        40,  45])

In [14]:
choices = np.array([ 11, 122,  37,  32,  70, 113,  77, 116, 134,  74, 143,  66,  41,
        81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
        40,  45])

In [ ]:
choices = np.array([11, 32, 36, 37, 40, 41, 45, 49, 51, 62, 65, 66, 70, 74, 76, 77, 80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])

In [30]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [20]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

11 (0.1, 0.5, 0.0001, 0.2)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 997ms/step - loss: 5.2262 - acc: 0.3015 - val_loss: 3.0273 - val_acc: 0.4736

Epoch 00001: val_acc improved from -inf to 0.47363, saving model to /data/oxford102/experiments/1546194827.202901/mobilenet_v2_1546194827.202901.h5
Epoch 2/100
17/17 [==============================] - 9s 553ms/step - loss: 2.4688 - acc: 0.5253 - val_loss: 2.0011 - val_acc: 0.5538

Epoch 00002: val_acc improved from 0.47363 to 0.55383, saving model to /data/oxford102/experiments/1546194827.202901/mobilenet_v2_1546194827.202901.h5
Epoch 3/100
17/17 [==============================] - 12s 678ms/step - loss: 1.0251 - acc: 0.7396 - val_loss: 1.7551 - val_acc: 0.6335

Epoch 00003: val_acc improved from 0.55383 to 0.63354, saving model to /data/oxford102/experiments/1546194827.202901/mobilenet_v2_1546194827.202901.h5
Epoch 4/100
17/17 [==============================] - 13s 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 984ms/step - loss: 2.4607 - acc: 0.4789 - val_loss: 1.4975 - val_acc: 0.6797

Epoch 00001: val_acc improved from -inf to 0.67969, saving model to /data/oxford102/experiments/1546195054.0470154/mobilenet_v2_1546195054.0470154.h5
Epoch 2/100
17/17 [==============================] - 9s 523ms/step - loss: 1.2338 - acc: 0.7298 - val_loss: 1.0619 - val_acc: 0.7774

Epoch 00002: val_acc improved from 0.67969 to 0.77743, saving model to /data/oxford102/experiments/1546195054.0470154/mobilenet_v2_1546195054.0470154.h5
Epoch 3/100
17/17 [==============================] - 13s 754ms/step - loss: 0.4001 - acc: 0.9454 - val_loss: 0.8685 - val_acc: 0.8168

Epoch 00003: val_acc improved from 0.77743 to 0.81677, saving model to /data/oxford102/experiments/1546195054.0470154/mobilenet_v2_1546195054.0470154.h5
Epoch 4/100
17/17 [==============================] -

Epoch 21/100
17/17 [==============================] - 13s 793ms/step - loss: 4.9723 - acc: 0.6696 - val_loss: 6.6277 - val_acc: 0.5435

Epoch 00021: val_acc did not improve from 0.57143
Epoch 22/100
17/17 [==============================] - 14s 807ms/step - loss: 4.7917 - acc: 0.6828 - val_loss: 6.3122 - val_acc: 0.5673

Epoch 00022: val_acc did not improve from 0.57143
Epoch 23/100
17/17 [==============================] - 14s 809ms/step - loss: 4.5135 - acc: 0.7036 - val_loss: 6.4622 - val_acc: 0.5559

Epoch 00023: val_acc did not improve from 0.57143
Epoch 00023: early stopping
{'network_name': 'mobilenet_v2', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546195246.9031267}
32 (0.1, 0.9, None, 0.001)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 13.8414 - acc: 0.0850 - va


Epoch 00001: val_acc improved from -inf to 0.68359, saving model to /data/oxford102/experiments/1546196202.6338403/mobilenet_v2_1546196202.6338403.h5
Epoch 2/100
17/17 [==============================] - 9s 502ms/step - loss: 1.1980 - acc: 0.7289 - val_loss: 1.0537 - val_acc: 0.7650

Epoch 00002: val_acc improved from 0.68359 to 0.76501, saving model to /data/oxford102/experiments/1546196202.6338403/mobilenet_v2_1546196202.6338403.h5
Epoch 3/100
17/17 [==============================] - 13s 749ms/step - loss: 0.3374 - acc: 0.9528 - val_loss: 0.8397 - val_acc: 0.8075

Epoch 00003: val_acc improved from 0.76501 to 0.80745, saving model to /data/oxford102/experiments/1546196202.6338403/mobilenet_v2_1546196202.6338403.h5
Epoch 4/100
17/17 [==============================] - 13s 742ms/step - loss: 0.2207 - acc: 0.9715 - val_loss: 0.7684 - val_acc: 0.8178

Epoch 00004: val_acc improved from 0.80745 to 0.81781, saving model to /data/oxford102/experiments/1546196202.6338403/mobilenet_v2_15461962

17/17 [==============================] - 14s 795ms/step - loss: 0.0768 - acc: 0.9949 - val_loss: 0.6220 - val_acc: 0.8261

Epoch 00008: val_acc improved from 0.80849 to 0.82609, saving model to /data/oxford102/experiments/1546197412.8965957/mobilenet_v2_1546197412.8965957.h5
Epoch 9/100
17/17 [==============================] - 13s 775ms/step - loss: 0.0695 - acc: 0.9936 - val_loss: 0.6792 - val_acc: 0.8106

Epoch 00009: val_acc did not improve from 0.82609
Epoch 10/100
17/17 [==============================] - 15s 891ms/step - loss: 0.0556 - acc: 0.9945 - val_loss: 0.6162 - val_acc: 0.8301

Epoch 00010: val_acc improved from 0.82609 to 0.83008, saving model to /data/oxford102/experiments/1546197412.8965957/mobilenet_v2_1546197412.8965957.h5
Epoch 11/100
17/17 [==============================] - 14s 818ms/step - loss: 0.0468 - acc: 0.9968 - val_loss: 0.6181 - val_acc: 0.8188

Epoch 00011: val_acc did not improve from 0.83008
Epoch 12/100
17/17 [==============================] - 13s 790ms/

ResourceExhaustedError: OOM when allocating tensor with shape[12288,113,113] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block_1_pad_10/Pad}} = Pad[T=DT_FLOAT, Tpaddings=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block_1_expand_relu_10/Relu6, block_1_pad_10/Pad-1-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_10/mul/_28463}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1648_loss_10/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [15]:
choices = np.array([143,  66,  41,
        81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
        40,  45])

In [18]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

143 (0.001, 0.9, 0.01, 0.2)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.9098 - acc: 0.3773 - val_loss: 2.0521 - val_acc: 0.5371

Epoch 00001: val_acc improved from -inf to 0.53711, saving model to /data/oxford102/experiments/1546200669.4405224/mobilenet_v2_1546200669.4405224.h5
Epoch 2/100
17/17 [==============================] - 9s 556ms/step - loss: 1.8471 - acc: 0.5800 - val_loss: 1.6486 - val_acc: 0.6346

Epoch 00002: val_acc improved from 0.53711 to 0.63458, saving model to /data/oxford102/experiments/1546200669.4405224/mobilenet_v2_1546200669.4405224.h5
Epoch 3/100
17/17 [==============================] - 12s 701ms/step - loss: 1.0086 - acc: 0.8040 - val_loss: 1.5499 - val_acc: 0.6439

Epoch 00003: val_acc improved from 0.63458 to 0.64389, saving model to /data/oxford102/experiments/1546200669.4405224/mobilenet_v2_1546200669.4405224.h5
Epoch 4/100
17/17 [==============================] - 14

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.5441 - acc: 0.4389 - val_loss: 1.5910 - val_acc: 0.6016

Epoch 00001: val_acc improved from -inf to 0.60156, saving model to /data/oxford102/experiments/1546200893.573037/mobilenet_v2_1546200893.573037.h5
Epoch 2/100
17/17 [==============================] - 9s 520ms/step - loss: 1.2389 - acc: 0.6847 - val_loss: 1.0585 - val_acc: 0.7319

Epoch 00002: val_acc improved from 0.60156 to 0.73188, saving model to /data/oxford102/experiments/1546200893.573037/mobilenet_v2_1546200893.573037.h5
Epoch 3/100
17/17 [==============================] - 12s 712ms/step - loss: 0.3513 - acc: 0.9257 - val_loss: 0.7838 - val_acc: 0.7743

Epoch 00003: val_acc improved from 0.73188 to 0.77433, saving model to /data/oxford102/experiments/1546200893.573037/mobilenet_v2_1546200893.573037.h5
Epoch 4/100
17/17 [==============================] - 13s 761m

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.4000 - acc: 0.4913 - val_loss: 1.4170 - val_acc: 0.6787

Epoch 00001: val_acc improved from -inf to 0.67871, saving model to /data/oxford102/experiments/1546201990.603477/mobilenet_v2_1546201990.603477.h5
Epoch 2/100
17/17 [==============================] - 9s 501ms/step - loss: 1.1521 - acc: 0.7564 - val_loss: 1.0202 - val_acc: 0.7743

Epoch 00002: val_acc improved from 0.67871 to 0.77433, saving model to /data/oxford102/experiments/1546201990.603477/mobilenet_v2_1546201990.603477.h5
Epoch 3/100
17/17 [==============================] - 12s 718ms/step - loss: 0.3337 - acc: 0.9546 - val_loss: 0.8129 - val_acc: 0.8137

Epoch 00003: val_acc improved from 0.77433 to 0.81366, saving model to /data/oxford102/experiments/1546201990.603477/mobilenet_v2_1546201990.603477.h5
Epoch 4/100
17/17 [==============================] - 13s 752m

17/17 [==============================] - 13s 759ms/step - loss: 0.0328 - acc: 1.0000 - val_loss: 0.6110 - val_acc: 0.8509

Epoch 00007: val_acc did not improve from 0.86232
Epoch 8/100
17/17 [==============================] - 13s 757ms/step - loss: 0.0234 - acc: 0.9991 - val_loss: 0.5918 - val_acc: 0.8582

Epoch 00008: val_acc did not improve from 0.86232
Epoch 9/100
17/17 [==============================] - 13s 792ms/step - loss: 0.0178 - acc: 1.0000 - val_loss: 0.5621 - val_acc: 0.8613

Epoch 00009: val_acc did not improve from 0.86232
Epoch 00009: early stopping
{'network_name': 'mobilenet_v2', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546203107.7764108}
80 (0.01, 0.9, None, 0.001)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.6597 - acc: 0.4242 - val_loss: 1.7230 

ResourceExhaustedError: OOM when allocating tensor with shape[12288,113,113] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block_1_pad_10/Pad}} = Pad[T=DT_FLOAT, Tpaddings=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block_1_expand_relu_10/Relu6, block_1_pad_10/Pad-1-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_10/mul/_28463}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1648_loss_10/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


# vgg19

In [15]:
params = {'network_name': "vgg19",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [16]:
choices = np.array([11, 32, 36, 37, 40, 41, 45, 49, 51, 62, 65, 66, 70, 74, 76, 77, 80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

11 (0.1, 0.5, 0.0001, 0.2)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 44s 3s/step - loss: 5.1475 - acc: 0.3355 - val_loss: 3.0476 - val_acc: 0.4971

Epoch 00001: val_acc improved from -inf to 0.49707, saving model to /data/oxford102/experiments/1546204734.0442073/vgg19_1546204734.0442073.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 2.1652 - acc: 0.5689 - val_loss: 1.7721 - val_acc: 0.6056

Epoch 00002: val_acc improved from 0.49707 to 0.60559, saving model to /data/oxford102/experiments/1546204734.0442073/vgg19_1546204734.0442073.h5
Epoch 3/100
17/17 [==============================] - 41s 2s/step - loss: 0.7591 - acc: 0.8192 - val_loss: 1.6301 - val_acc: 0.6636

Epoch 00003: val_acc improved from 0.60559 to 0.66356, saving model to /data/oxford102/experiments/1546204734.0442073/vgg19_1546204734.0442073.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.5354 -

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 14.1866 - acc: 0.0657 - val_loss: 13.6025 - val_acc: 0.1387

Epoch 00001: val_acc improved from -inf to 0.13867, saving model to /data/oxford102/experiments/1546205279.262386/vgg19_1546205279.262386.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 13.1396 - acc: 0.1668 - val_loss: 12.9301 - val_acc: 0.1853

Epoch 00002: val_acc improved from 0.13867 to 0.18530, saving model to /data/oxford102/experiments/1546205279.262386/vgg19_1546205279.262386.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 12.4066 - acc: 0.2131 - val_loss: 12.7615 - val_acc: 0.1936

Epoch 00003: val_acc improved from 0.18530 to 0.19358, saving model to /data/oxford102/experiments/1546205279.262386/vgg19_1546205279.262386.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 12.4156 

Epoch 5/100
17/17 [==============================] - 33s 2s/step - loss: 12.3440 - acc: 0.2210 - val_loss: 12.3614 - val_acc: 0.2205

Epoch 00005: val_acc improved from 0.19462 to 0.22050, saving model to /data/oxford102/experiments/1546207289.1490753/vgg19_1546207289.1490753.h5
Epoch 6/100
17/17 [==============================] - 33s 2s/step - loss: 12.1579 - acc: 0.2344 - val_loss: 12.6371 - val_acc: 0.2081

Epoch 00006: val_acc did not improve from 0.22050
Epoch 7/100
17/17 [==============================] - 33s 2s/step - loss: 12.0345 - acc: 0.2455 - val_loss: 12.5876 - val_acc: 0.2060

Epoch 00007: val_acc did not improve from 0.22050
Epoch 8/100
17/17 [==============================] - 33s 2s/step - loss: 11.7127 - acc: 0.2638 - val_loss: 12.0078 - val_acc: 0.2402

Epoch 00008: val_acc improved from 0.22050 to 0.24017, saving model to /data/oxford102/experiments/1546207289.1490753/vgg19_1546207289.1490753.h5
Epoch 9/100
17/17 [==============================] - 33s 2s/step - loss:

17/17 [==============================] - 33s 2s/step - loss: 6.6506 - acc: 0.5723 - val_loss: 7.9587 - val_acc: 0.4710

Epoch 00024: val_acc did not improve from 0.48965
Epoch 25/100
17/17 [==============================] - 33s 2s/step - loss: 6.2577 - acc: 0.5896 - val_loss: 7.6554 - val_acc: 0.4855

Epoch 00025: val_acc did not improve from 0.48965
Epoch 26/100
17/17 [==============================] - 33s 2s/step - loss: 6.2142 - acc: 0.5984 - val_loss: 7.6012 - val_acc: 0.4824

Epoch 00026: val_acc did not improve from 0.48965
Epoch 00026: early stopping
{'network_name': 'vgg19', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546207738.787315}
45 (0.1, 0.9, 0.01, 0.01)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 36s 2s/step - loss: 5.1984 - acc: 0.2785 - val_loss: 3.2462 - val_acc: 0.4014


Epoch 00005: val_acc improved from 0.78882 to 0.80124, saving model to /data/oxford102/experiments/1546209444.510648/vgg19_1546209444.510648.h5
Epoch 6/100
17/17 [==============================] - 33s 2s/step - loss: 0.0549 - acc: 0.9963 - val_loss: 0.7479 - val_acc: 0.8106

Epoch 00006: val_acc improved from 0.80124 to 0.81056, saving model to /data/oxford102/experiments/1546209444.510648/vgg19_1546209444.510648.h5
Epoch 7/100
17/17 [==============================] - 33s 2s/step - loss: 0.0299 - acc: 0.9991 - val_loss: 0.7495 - val_acc: 0.8157

Epoch 00007: val_acc improved from 0.81056 to 0.81573, saving model to /data/oxford102/experiments/1546209444.510648/vgg19_1546209444.510648.h5
Epoch 8/100
17/17 [==============================] - 34s 2s/step - loss: 0.0225 - acc: 1.0000 - val_loss: 0.7997 - val_acc: 0.7961

Epoch 00008: val_acc did not improve from 0.81573
Epoch 9/100
17/17 [==============================] - 33s 2s/step - loss: 0.0161 - acc: 1.0000 - val_loss: 0.7951 - val_ac


Epoch 00015: val_acc did not improve from 0.78675
Epoch 16/100
17/17 [==============================] - 34s 2s/step - loss: 0.0059 - acc: 1.0000 - val_loss: 1.0933 - val_acc: 0.7619

Epoch 00016: val_acc did not improve from 0.78675
Epoch 17/100
17/17 [==============================] - 34s 2s/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.9877 - val_acc: 0.7733

Epoch 00017: val_acc did not improve from 0.78675
Epoch 00017: early stopping
{'network_name': 'vgg19', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546210222.7773154}
66 (0.01, 0.75, None, 0.1)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 37s 2s/step - loss: 2.4550 - acc: 0.4628 - val_loss: 1.4668 - val_acc: 0.6445

Epoch 00001: val_acc improved from -inf to 0.64453, saving model to /data/oxford102/experiments/1546210816.6303937/

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2_12/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1_12/Relu, block1_conv2_12/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_11/mul/_4893}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_845_loss_11/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add re

In [14]:
choices = np.array([70, 74, 76, 77, 80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])

In [31]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

70 (0.01, 0.75, 1e-06, 0.1)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.5611 - acc: 0.4375 - val_loss: 1.5957 - val_acc: 0.6035

Epoch 00001: val_acc improved from -inf to 0.60352, saving model to /data/oxford102/experiments/1546218768.0712278/mobilenet_v2_1546218768.0712278.h5
Epoch 2/100
17/17 [==============================] - 10s 563ms/step - loss: 1.2338 - acc: 0.6733 - val_loss: 1.0490 - val_acc: 0.7236

Epoch 00002: val_acc improved from 0.60352 to 0.72360, saving model to /data/oxford102/experiments/1546218768.0712278/mobilenet_v2_1546218768.0712278.h5
Epoch 3/100
17/17 [==============================] - 12s 710ms/step - loss: 0.3484 - acc: 0.9210 - val_loss: 0.8109 - val_acc: 0.7764

Epoch 00003: val_acc improved from 0.72360 to 0.77640, saving model to /data/oxford102/experiments/1546218768.0712278/mobilenet_v2_1546218768.0712278.h5
Epoch 4/100
17/17 [==============================] - 1

ResourceExhaustedError: OOM when allocating tensor with shape[12288,113,113] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block_1_pad_4/Pad}} = Pad[T=DT_FLOAT, Tpaddings=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block_1_expand_relu_4/Relu6, block_1_pad_4/Pad-1-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [14]:
choices = np.array([80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])


In [17]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

80 (0.01, 0.9, None, 0.001)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 44s 3s/step - loss: 2.6755 - acc: 0.4288 - val_loss: 1.5770 - val_acc: 0.6055

Epoch 00001: val_acc improved from -inf to 0.60547, saving model to /data/oxford102/experiments/1546220311.4551632/vgg19_1546220311.4551632.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 1.3281 - acc: 0.6517 - val_loss: 1.2252 - val_acc: 0.6843

Epoch 00002: val_acc improved from 0.60547 to 0.68427, saving model to /data/oxford102/experiments/1546220311.4551632/vgg19_1546220311.4551632.h5
Epoch 3/100
17/17 [==============================] - 42s 2s/step - loss: 0.3833 - acc: 0.9063 - val_loss: 1.0783 - val_acc: 0.7526

Epoch 00003: val_acc improved from 0.68427 to 0.75259, saving model to /data/oxford102/experiments/1546220311.4551632/vgg19_1546220311.4551632.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.2722 -

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 2.7317 - acc: 0.4338 - val_loss: 1.6108 - val_acc: 0.6064

Epoch 00001: val_acc improved from -inf to 0.60645, saving model to /data/oxford102/experiments/1546220686.6024904/vgg19_1546220686.6024904.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.2747 - acc: 0.6659 - val_loss: 1.1983 - val_acc: 0.6967

Epoch 00002: val_acc improved from 0.60645 to 0.69669, saving model to /data/oxford102/experiments/1546220686.6024904/vgg19_1546220686.6024904.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 0.3205 - acc: 0.9238 - val_loss: 1.0213 - val_acc: 0.7360

Epoch 00003: val_acc improved from 0.69669 to 0.73602, saving model to /data/oxford102/experiments/1546220686.6024904/vgg19_1546220686.6024904.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.1860 -

17/17 [==============================] - 33s 2s/step - loss: 0.0169 - acc: 0.9995 - val_loss: 0.7443 - val_acc: 0.8230

Epoch 00007: val_acc improved from 0.80642 to 0.82298, saving model to /data/oxford102/experiments/1546222563.630598/vgg19_1546222563.630598.h5
Epoch 8/100
17/17 [==============================] - 33s 2s/step - loss: 0.0103 - acc: 1.0000 - val_loss: 0.7558 - val_acc: 0.8137

Epoch 00008: val_acc did not improve from 0.82298
Epoch 9/100
17/17 [==============================] - 33s 2s/step - loss: 0.0066 - acc: 1.0000 - val_loss: 0.7088 - val_acc: 0.8292

Epoch 00009: val_acc improved from 0.82298 to 0.82919, saving model to /data/oxford102/experiments/1546222563.630598/vgg19_1546222563.630598.h5
Epoch 10/100
17/17 [==============================] - 33s 2s/step - loss: 0.0037 - acc: 1.0000 - val_loss: 0.7621 - val_acc: 0.8232

Epoch 00010: val_acc did not improve from 0.82919
Epoch 11/100
17/17 [==============================] - 33s 2s/step - loss: 0.0028 - acc: 1.0000 

Epoch 8/100
17/17 [==============================] - 33s 2s/step - loss: 0.0737 - acc: 0.9995 - val_loss: 0.9559 - val_acc: 0.7671

Epoch 00008: val_acc did not improve from 0.77950
Epoch 9/100
17/17 [==============================] - 33s 2s/step - loss: 0.0675 - acc: 0.9972 - val_loss: 0.9796 - val_acc: 0.7650

Epoch 00009: val_acc did not improve from 0.77950
Epoch 10/100
17/17 [==============================] - 34s 2s/step - loss: 0.0503 - acc: 1.0000 - val_loss: 0.9685 - val_acc: 0.7744

Epoch 00010: val_acc did not improve from 0.77950
Epoch 00010: early stopping
{'network_name': 'vgg19', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546223612.0649786}
134 (0.001, 0.9, 1e-06, 0.1)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 2.5331 - acc: 0.4747 - val_loss: 1.6855 - 

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2_11/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1_11/Relu, block1_conv2_11/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_11/acc/Mean/_4679}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_848_metrics_11/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM

In [18]:
choices = np.array([143])

In [19]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

143 (0.001, 0.9, 0.01, 0.2)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 43s 3s/step - loss: 3.5952 - acc: 0.2564 - val_loss: 2.8888 - val_acc: 0.3809

Epoch 00001: val_acc improved from -inf to 0.38086, saving model to /data/oxford102/experiments/1546225434.4993954/vgg19_1546225434.4993954.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 2.7367 - acc: 0.3915 - val_loss: 2.4865 - val_acc: 0.4389

Epoch 00002: val_acc improved from 0.38086 to 0.43892, saving model to /data/oxford102/experiments/1546225434.4993954/vgg19_1546225434.4993954.h5
Epoch 3/100
17/17 [==============================] - 41s 2s/step - loss: 1.8021 - acc: 0.6194 - val_loss: 2.3347 - val_acc: 0.4638

Epoch 00003: val_acc improved from 0.43892 to 0.46377, saving model to /data/oxford102/experiments/1546225434.4993954/vgg19_1546225434.4993954.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 1.6313 -

In [18]:
params = {'network_name': "vgg19",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [19]:
choices = np.array([44, 60, 66, 164])

In [20]:
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

44 (0.1, 0.9, 0.01, 0.001)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 44s 3s/step - loss: 5.6126 - acc: 0.2491 - val_loss: 4.2003 - val_acc: 0.3477

Epoch 00001: val_acc improved from -inf to 0.34766, saving model to /data/oxford102/experiments/1546747745.64547/vgg19_1546747745.64547.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 3.1729 - acc: 0.4416 - val_loss: 2.4740 - val_acc: 0.5031

Epoch 00002: val_acc improved from 0.34766 to 0.50311, saving model to /data/oxford102/experiments/1546747745.64547/vgg19_1546747745.64547.h5
Epoch 3/100
17/17 [==============================] - 42s 2s/step - loss: 1.6608 - acc: 0.6278 - val_loss: 1.8903 - val_acc: 0.5911

Epoch 00003: val_acc improved from 0.50311 to 0.59110, saving model to /data/oxford102/experiments/1546747745.64547/vgg19_1546747745.64547.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 1.4534 - acc: 0.6677

Epoch 7/100
17/17 [==============================] - 33s 2s/step - loss: 0.0209 - acc: 0.9991 - val_loss: 0.7428 - val_acc: 0.8137

Epoch 00007: val_acc did not improve from 0.82091
Epoch 8/100
17/17 [==============================] - 33s 2s/step - loss: 0.0123 - acc: 0.9995 - val_loss: 0.7657 - val_acc: 0.8054

Epoch 00008: val_acc did not improve from 0.82091
Epoch 9/100
17/17 [==============================] - 33s 2s/step - loss: 0.0100 - acc: 0.9991 - val_loss: 0.7084 - val_acc: 0.8333

Epoch 00009: val_acc improved from 0.82091 to 0.83333, saving model to /data/oxford102/experiments/1546748609.9073424/vgg19_1546748609.9073424.h5
Epoch 10/100
17/17 [==============================] - 33s 2s/step - loss: 0.0067 - acc: 1.0000 - val_loss: 0.7012 - val_acc: 0.8301

Epoch 00010: val_acc did not improve from 0.83333
Epoch 11/100
17/17 [==============================] - 33s 2s/step - loss: 0.0047 - acc: 1.0000 - val_loss: 0.6998 - val_acc: 0.8230

Epoch 00011: val_acc did not improve from 

IndexError: list index out of range

# Remaining options Mobilenet

In [15]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [20]:
choices = np.array([36, 45, 65, 91, 98, 135, 164])
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

36 (0.1, 0.9, 1e-06, 0.001)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 21s 1s/step - loss: 13.9542 - acc: 0.0832 - val_loss: 13.7382 - val_acc: 0.1367

Epoch 00001: val_acc improved from -inf to 0.13672, saving model to /data/oxford102/experiments/1546739785.05324/mobilenet_v2_1546739785.05324.h5
Epoch 2/100
17/17 [==============================] - 11s 669ms/step - loss: 13.7181 - acc: 0.1392 - val_loss: 13.0940 - val_acc: 0.1812

Epoch 00002: val_acc improved from 0.13672 to 0.18116, saving model to /data/oxford102/experiments/1546739785.05324/mobilenet_v2_1546739785.05324.h5
Epoch 3/100
17/17 [==============================] - 15s 867ms/step - loss: 12.7134 - acc: 0.1980 - val_loss: 12.8253 - val_acc: 0.1925

Epoch 00003: val_acc improved from 0.18116 to 0.19255, saving model to /data/oxford102/experiments/1546739785.05324/mobilenet_v2_1546739785.05324.h5
Epoch 4/100
17/17 [==============================] - 15s 873

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 19s 1s/step - loss: 4.9749 - acc: 0.2647 - val_loss: 3.4083 - val_acc: 0.3838

Epoch 00001: val_acc improved from -inf to 0.38379, saving model to /data/oxford102/experiments/1546740106.328966/mobilenet_v2_1546740106.328966.h5
Epoch 2/100
17/17 [==============================] - 9s 552ms/step - loss: 2.7308 - acc: 0.4315 - val_loss: 2.1693 - val_acc: 0.5176

Epoch 00002: val_acc improved from 0.38379 to 0.51760, saving model to /data/oxford102/experiments/1546740106.328966/mobilenet_v2_1546740106.328966.h5
Epoch 3/100
17/17 [==============================] - 13s 752ms/step - loss: 1.4961 - acc: 0.6229 - val_loss: 1.7485 - val_acc: 0.5704

Epoch 00003: val_acc improved from 0.51760 to 0.57039, saving model to /data/oxford102/experiments/1546740106.328966/mobilenet_v2_1546740106.328966.h5
Epoch 4/100
17/17 [==============================] - 14s 836m

Epoch 3/100
17/17 [==============================] - 13s 758ms/step - loss: 0.4712 - acc: 0.8805 - val_loss: 0.9057 - val_acc: 0.7619

Epoch 00003: val_acc improved from 0.70600 to 0.76190, saving model to /data/oxford102/experiments/1546740651.5176127/mobilenet_v2_1546740651.5176127.h5
Epoch 4/100
17/17 [==============================] - 13s 790ms/step - loss: 0.3732 - acc: 0.9017 - val_loss: 0.8366 - val_acc: 0.7795

Epoch 00004: val_acc improved from 0.76190 to 0.77950, saving model to /data/oxford102/experiments/1546740651.5176127/mobilenet_v2_1546740651.5176127.h5
Epoch 5/100
17/17 [==============================] - 13s 754ms/step - loss: 0.2030 - acc: 0.9487 - val_loss: 0.8480 - val_acc: 0.7650

Epoch 00005: val_acc did not improve from 0.77950
Epoch 6/100
17/17 [==============================] - 14s 811ms/step - loss: 0.1186 - acc: 0.9729 - val_loss: 0.8006 - val_acc: 0.7940

Epoch 00006: val_acc improved from 0.77950 to 0.79400, saving model to /data/oxford102/experiments/15467


Epoch 00003: val_acc improved from 0.80952 to 0.82298, saving model to /data/oxford102/experiments/1546741254.0010242/mobilenet_v2_1546741254.0010242.h5
Epoch 4/100
17/17 [==============================] - 15s 880ms/step - loss: 0.1968 - acc: 0.9798 - val_loss: 0.7250 - val_acc: 0.8385

Epoch 00004: val_acc improved from 0.82298 to 0.83851, saving model to /data/oxford102/experiments/1546741254.0010242/mobilenet_v2_1546741254.0010242.h5
Epoch 5/100
17/17 [==============================] - 14s 819ms/step - loss: 0.0895 - acc: 0.9949 - val_loss: 0.6872 - val_acc: 0.8313

Epoch 00005: val_acc did not improve from 0.83851
Epoch 6/100
17/17 [==============================] - 14s 853ms/step - loss: 0.0540 - acc: 0.9995 - val_loss: 0.6454 - val_acc: 0.8582

Epoch 00006: val_acc improved from 0.83851 to 0.85818, saving model to /data/oxford102/experiments/1546741254.0010242/mobilenet_v2_1546741254.0010242.h5
Epoch 7/100
17/17 [==============================] - 14s 827ms/step - loss: 0.0371 - 

IndexError: list index out of range

In [17]:
params['network_name'] = "mobilenet_v2"
choices = np.array([76])
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

76 (0.01, 0.75, 0.01, 0.001)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.7026 - acc: 0.4062 - val_loss: 1.7612 - val_acc: 0.5752

Epoch 00001: val_acc improved from -inf to 0.57520, saving model to /data/oxford102/experiments/1546743469.5575337/mobilenet_v2_1546743469.5575337.h5
Epoch 2/100
17/17 [==============================] - 10s 578ms/step - loss: 1.4191 - acc: 0.6402 - val_loss: 1.1794 - val_acc: 0.6925

Epoch 00002: val_acc improved from 0.57520 to 0.69255, saving model to /data/oxford102/experiments/1546743469.5575337/mobilenet_v2_1546743469.5575337.h5
Epoch 3/100
17/17 [==============================] - 13s 768ms/step - loss: 0.6092 - acc: 0.8631 - val_loss: 0.8698 - val_acc: 0.7660

Epoch 00003: val_acc improved from 0.69255 to 0.76605, saving model to /data/oxford102/experiments/1546743469.5575337/mobilenet_v2_1546743469.5575337.h5
Epoch 4/100
17/17 [==============================] - 1

# inception_resnet_v2

In [20]:
params = {'network_name': "inception_resnet_v2",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [21]:
choices = np.array([11, 32, 36, 37, 40, 41, 45, 49, 51, 62, 65, 66, 70, 74, 76, 77, 80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])
for i in choices:
    print(i, grid[i])
    lr, rho, epsilon, decay = grid[i]
    _optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
    params['optimizer'] = _optimizer
    _model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
    train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
    train_model(params, _model, (train_generator, validation_generator))

11 (0.1, 0.5, 0.0001, 0.2)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 45s 3s/step - loss: 7.0371 - acc: 0.1558 - val_loss: 4.9081 - val_acc: 0.2295

Epoch 00001: val_acc improved from -inf to 0.22949, saving model to /data/oxford102/experiments/1546226115.7110944/inception_resnet_v2_1546226115.7110944.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 4.1851 - acc: 0.2909 - val_loss: 3.5349 - val_acc: 0.3199

Epoch 00002: val_acc improved from 0.22949 to 0.31988, saving model to /data/oxford102/experiments/1546226115.7110944/inception_resnet_v2_1546226115.7110944.h5
Epoch 3/100
17/17 [==============================] - 39s 2s/step - loss: 2.6752 - acc: 0.4133 - val_loss: 3.0221 - val_acc: 0.3716

Epoch 00003: val_acc improved from 0.31988 to 0.37164, saving model to /data/oxford102/experiments/1546226115.7110944/inception_resnet_v2_1546226115.7110944.h5
Epoch 4/100
17/17 [====================


Epoch 00002: val_acc improved from 0.04590 to 0.04762, saving model to /data/oxford102/experiments/1546227487.1148376/inception_resnet_v2_1546227487.1148376.h5
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 15.2944 - acc: 0.0488 - val_loss: 15.2317 - val_acc: 0.0518

Epoch 00003: val_acc improved from 0.04762 to 0.05176, saving model to /data/oxford102/experiments/1546227487.1148376/inception_resnet_v2_1546227487.1148376.h5
Epoch 4/100
17/17 [==============================] - 36s 2s/step - loss: 15.0518 - acc: 0.0620 - val_loss: 15.0982 - val_acc: 0.0590

Epoch 00004: val_acc improved from 0.05176 to 0.05901, saving model to /data/oxford102/experiments/1546227487.1148376/inception_resnet_v2_1546227487.1148376.h5
Epoch 5/100
17/17 [==============================] - 36s 2s/step - loss: 15.2301 - acc: 0.0530 - val_loss: 15.3005 - val_acc: 0.0476

Epoch 00005: val_acc did not improve from 0.05901
Epoch 6/100
17/17 [==============================] - 36s 2s/step - 

Epoch 29/100
17/17 [==============================] - 37s 2s/step - loss: 1.8306 - acc: 0.7413 - val_loss: 3.4468 - val_acc: 0.5518

Epoch 00029: val_acc did not improve from 0.57129
Epoch 30/100
17/17 [==============================] - 37s 2s/step - loss: 1.7977 - acc: 0.7432 - val_loss: 3.2221 - val_acc: 0.5663

Epoch 00030: val_acc did not improve from 0.57129
Epoch 31/100
17/17 [==============================] - 37s 2s/step - loss: 1.4376 - acc: 0.7652 - val_loss: 2.9849 - val_acc: 0.5973

Epoch 00031: val_acc improved from 0.57129 to 0.59731, saving model to /data/oxford102/experiments/1546229506.2987907/inception_resnet_v2_1546229506.2987907.h5
Epoch 32/100
17/17 [==============================] - 37s 2s/step - loss: 1.3406 - acc: 0.7813 - val_loss: 3.2642 - val_acc: 0.5725

Epoch 00032: val_acc did not improve from 0.59731
Epoch 33/100
17/17 [==============================] - 37s 2s/step - loss: 1.0849 - acc: 0.8189 - val_loss: 2.7024 - val_acc: 0.6128

Epoch 00033: val_acc impr


Epoch 00014: val_acc did not improve from 0.19358
Epoch 00014: early stopping
{'network_name': 'inception_resnet_v2', 'image_aug': False, 'optimizer_name': 'RMSProp', 'optimizer_params': None, 'batch_size': 128, 'epochs': 100, 'image_size': (224, 224), 'log_time': 1546231979.2862241}
41 (0.1, 0.9, 0.0001, 0.01)
Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 45s 3s/step - loss: 14.4535 - acc: 0.0515 - val_loss: 14.9604 - val_acc: 0.0674

Epoch 00001: val_acc improved from -inf to 0.06738, saving model to /data/oxford102/experiments/1546233215.909061/inception_resnet_v2_1546233215.909061.h5
Epoch 2/100
17/17 [==============================] - 36s 2s/step - loss: 14.9164 - acc: 0.0657 - val_loss: 14.0869 - val_acc: 0.1046

Epoch 00002: val_acc improved from 0.06738 to 0.10455, saving model to /data/oxford102/experiments/1546233215.909061/inception_resnet_v2_1546233215.909061.h5
Epoch 3/100
17/17

Epoch 13/100
17/17 [==============================] - 37s 2s/step - loss: 0.9780 - acc: 0.7067 - val_loss: 1.7099 - val_acc: 0.5652

Epoch 00013: val_acc did not improve from 0.57867
Epoch 14/100
17/17 [==============================] - 37s 2s/step - loss: 0.8473 - acc: 0.7422 - val_loss: 1.5512 - val_acc: 0.5973

Epoch 00014: val_acc improved from 0.57867 to 0.59731, saving model to /data/oxford102/experiments/1546234653.0273695/inception_resnet_v2_1546234653.0273695.h5
Epoch 15/100
17/17 [==============================] - 37s 2s/step - loss: 0.8344 - acc: 0.7495 - val_loss: 1.5526 - val_acc: 0.6046

Epoch 00015: val_acc improved from 0.59731 to 0.60455, saving model to /data/oxford102/experiments/1546234653.0273695/inception_resnet_v2_1546234653.0273695.h5
Epoch 16/100
17/17 [==============================] - 37s 2s/step - loss: 0.7517 - acc: 0.7753 - val_loss: 1.5853 - val_acc: 0.5983

Epoch 00016: val_acc did not improve from 0.60455
Epoch 17/100
17/17 [============================

17/17 [==============================] - 37s 2s/step - loss: 0.9543 - acc: 0.7877 - val_loss: 1.4812 - val_acc: 0.6299

Epoch 00010: val_acc did not improve from 0.63354
Epoch 11/100
17/17 [==============================] - 37s 2s/step - loss: 0.9393 - acc: 0.8029 - val_loss: 1.4488 - val_acc: 0.6377

Epoch 00011: val_acc improved from 0.63354 to 0.63768, saving model to /data/oxford102/experiments/1546240195.3612895/inception_resnet_v2_1546240195.3612895.h5
Epoch 12/100
17/17 [==============================] - 37s 2s/step - loss: 0.8396 - acc: 0.8254 - val_loss: 1.3580 - val_acc: 0.6636

Epoch 00012: val_acc improved from 0.63768 to 0.66356, saving model to /data/oxford102/experiments/1546240195.3612895/inception_resnet_v2_1546240195.3612895.h5
Epoch 13/100
17/17 [==============================] - 37s 2s/step - loss: 0.8415 - acc: 0.8245 - val_loss: 1.4544 - val_acc: 0.6366

Epoch 00013: val_acc did not improve from 0.66356
Epoch 14/100
17/17 [==============================] - 37s 2s/

Epoch 2/100
17/17 [==============================] - 36s 2s/step - loss: 2.3548 - acc: 0.4260 - val_loss: 1.9676 - val_acc: 0.5114

Epoch 00002: val_acc improved from 0.38379 to 0.51139, saving model to /data/oxford102/experiments/1546244404.5197992/inception_resnet_v2_1546244404.5197992.h5
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 1.3935 - acc: 0.6466 - val_loss: 1.5968 - val_acc: 0.6056

Epoch 00003: val_acc improved from 0.51139 to 0.60559, saving model to /data/oxford102/experiments/1546244404.5197992/inception_resnet_v2_1546244404.5197992.h5
Epoch 4/100
17/17 [==============================] - 36s 2s/step - loss: 1.1108 - acc: 0.7082 - val_loss: 1.4184 - val_acc: 0.6222

Epoch 00004: val_acc improved from 0.60559 to 0.62215, saving model to /data/oxford102/experiments/1546244404.5197992/inception_resnet_v2_1546244404.5197992.h5
Epoch 5/100
17/17 [==============================] - 37s 2s/step - loss: 0.7252 - acc: 0.8364 - val_loss: 1.2652 - val_acc: 

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,109,109] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_2439/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](activation_2438/Relu, conv2d_2439/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_14/mul/_115133}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4592_loss_14/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_